# Module 5: Assignment - Comparison Exercise - Analyzing Deep Learning Frameworks

In [ ]:
%pip install kaggle

In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

# Download the bird identification dataset from Kaggle https://www.kaggle.com/datasets/gpiosenka/100-bird-species
dataset_name = 'gpiosenka/100-bird-species'
path_to_download = 'birds'  # Specify your download path here

api.dataset_download_files(dataset_name, path=path_to_download, unzip=True)

print(f'Dataset downloaded and extracted to: {path_to_download}')


### Training with the Kaggle 100 Bird Species dataset using TensorFlow

In [ ]:
# Import Tensor Flow
import tensorflow
# Import Keras
from tensorflow import keras
# Import Numpy
import numpy as np
# Import Matplotlib

# import the dataset
train_data = keras.utils.image_dataset_from_directory('birds/train', image_size=(224, 224), batch_size=32)
test_data = keras.utils.image_dataset_from_directory('birds/test', image_size=(224, 224), batch_size=32)
validation_data = keras.utils.image_dataset_from_directory('birds/valid', image_size=(224, 224), batch_size=32)
# Load the class names from csv file
class_names = np.loadtxt('birds/birds.csv', delimiter=',', usecols=(1,), dtype=str, skiprows=1)
# Print the class names
print(class_names)
# Print the shape of the training data
print(train_data)
# Print the shape of the test data
print(test_data)
# Print the shape of the validation data
print(validation_data)


# Example of normalization for image data
train_data = train_data.map(lambda x, y: (x / 255.0, y))
test_data = test_data.map(lambda x, y: (x / 255.0, y))
validation_data = validation_data.map(lambda x, y: (x / 255.0, y))

# Create a CNN model
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(525, activation='softmax')
])
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Fit the model
model.fit(train_data, epochs=10, validation_data=validation_data)

# Evaluate the model
model.evaluate(test_data)

# Save the model
model.save('birds_model')

# Load the model
loaded_model = keras.models.load_model('birds_model')

# Make predictions
predictions = loaded_model.predict(test_data)
# Print the predictionsf
print(predictions)


### Training with the 100 Bird Species dataset using PyTorch

In [3]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np


# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load datasets
train_dataset = datasets.ImageFolder('birds/train', transform=transform)
test_dataset = datasets.ImageFolder('birds/test', transform=transform)
validation_dataset = datasets.ImageFolder('birds/valid', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

# Load the class names from csv file
class_names = np.loadtxt('birds/birds.csv', delimiter=',', usecols=(1,), dtype=str, skiprows=1)
# Print the class names
print(class_names)
# Print the shape of the training data
print(train_dataset)
# Print the shape of the test data
print(test_dataset)
# Print the shape of the validation data
print(validation_dataset)


# Creating a temporary model to determine input size for first linear layer
temp_model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 32, 3, 1),
    torch.nn.MaxPool2d(2),
    torch.nn.Flatten()
)

x, _ = next(iter(train_loader))
x = temp_model(x)
print(x.shape)


# Create a CNN model with PyTorch
model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 32, 3, 1),
    torch.nn.MaxPool2d(2),
    torch.nn.Flatten(),
    torch.nn.Linear(394272, 128),
    torch.nn.Linear(128, len(class_names))
)

# move the model to the GPU
model.to(device)


# Define the loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters())

# Train the model
for epoch in range(10):
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        _, predicted = torch.max(y_pred, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

print(f'Accuracy: {correct / total}')


cuda
['train/ABBOTTS BABBLER/001.jpg' 'train/ABBOTTS BABBLER/007.jpg'
 'train/ABBOTTS BABBLER/008.jpg' ... 'valid/BLACK BREASTED PUFFBIRD/1.jpg'
 'valid/BLACK BREASTED PUFFBIRD/2.jpg'
 'valid/BLACK BREASTED PUFFBIRD/5.jpg']
Dataset ImageFolder
    Number of datapoints: 84635
    Root location: birds/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 2625
    Root location: birds/test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 2625
    Root location: birds/valid
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
torch